## [실습] HuggingFace에서 공개 모델 사용하기

http://huggingface.co 의 링크에서 공개된 모델을 사용하는 방법을 알아보겠습니다.   
큰 모델은 양자화를 통해 불러올 수 있습니다.

In [1]:
!pip install peft accelerate datasets huggingface_hub trl transformers bitsandbytes

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/4c/37/22ef7675bef4ffe9577b937ddca2e22791534cbbe11c30714972a91532dc/datasets-3.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/37/1f29af63e9c30156a3ed6ebc2754077016577c094f31de7b2631e5d379eb/transformers-4.49.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
  Obtaining dependency information for tokenizers<0.22,>=0.21 from https://files.pythonhosted.org/packages/22/7a/88e58bb297c22633ed1c9d16029316e5b5ac5ee44012164c2edede599a5e/tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 36.0 MB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 33.2 MB/s eta 0:00:0000:0100:01
  Attempting uninsta

In [2]:
!pip install langchain langchain-community langchain-huggingface

허깅페이스 모델에 접속하기 위해, 허깅페이스 라이브러리를 불러옵니다.

In [ ]:
from huggingface_hub import login
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path("/Users/blueno/UNO/SKALA/SKALA/.env")
load_dotenv(dotenv_path=env_path, override=True)

# 환경 변수 확인
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")

login(token=huggingface_token) # API 키를 입력합니다.

모델과 토크나이저를 불러옵니다.   
토크나이저는 모델의 구동에 필수적이며, 모델별 Special Token이나 Chat Template 등의 요소가 포함되어 있습니다.    
따라서 모델을 저장할 때는 토크나이저도 항상 같이 저장해주어야 합니다.

In [17]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [5]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True
)

In [6]:
import transformers
from transformers import pipeline
from transformers import AutoModelForCausalLM,AutoTokenizer

# 모델의 주소를 입력합니다.
# model_id = "Qwen/Qwen2.5-1.5B-Instruct" # 각 모델들을 순서대로 각각 불러와서 설치
# model_id = "Qwen/Qwen2.5-1.5B"
# model_id = "meta-llama/Llama-3.2-1B"
model_id = "meta-llama/Llama-3.2-1B-instruct"

# huggingface.co/ 뒤에 붙는 주소와 동일합니다.


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,

    torch_dtype='auto', # 가중치의 데이터타입 자동 감지

    # quantization_config=quantization_config,
    # 양자화란? 큰 모델의 가중치를 작게 만드는 방법

    # 32비트 모델은 *4, 16비트 모델은 *2를 하여 가용 파라미터 확인

    # 이미 양자화된 모델을 불러올 때는
    # config.json에 양자화 옵션이 들어가기 때문에
    # quantization_config을 안 넣어도 됨

    device_map={"":0}) # 0번 GPU에 할당

# 한번 불러온 모델은 .cache에 저장되어 이후 바로 불러올 수 있음


모델의 구조는 아래와 같이 확인 가능합니다.

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [8]:
model.save_pretrained("llama_test")

In [9]:
tokenizer.save_pretrained("llma_test")

('llma_test/tokenizer_config.json',
 'llma_test/special_tokens_map.json',
 'llma_test/tokenizer.json')

모델의 Generation을 잘 수행하기 위해, *text-generation* pipeline을 사용합니다.    
기존의 모델에 들어가던 매개변수(max_new_tokens 등)은 여기에서 포함합니다.

In [10]:
# 아래 옵션은 주로 모델 페이지의 generate_config에서 참고할 수 있습니다.
gen_config = dict(
    do_sample=True,
    # do_sample : 확률 샘플링의 영부 (False: Greedy)
    max_new_tokens=512,
    repetition_penalty = 1.1,
    # 동일 토큰 반복 방지 패널티
    temperature = 0.7,
    top_p = 0.8,
    # 확률 상위 80% 안에서만 샘플링
    top_k = 20
    # Top 20안에서만 샘플링 --> P와 K 교집합으로 처리됨
)


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    **gen_config
    )
# return_full_text : 프롬프트를 포함하여 출력할 것인지 결정 (Default : True)

Device set to use mps:0


In [11]:
pipe("거대 언어 모델이 뭐야?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '?\n거대 언어 모델 (Giant Language Model)란 단어, 문장을 조작하여 새로운 문장, 단어, 또는 문구를 생성하는 텍스트生成체입니다. \n어떤 개념이나 문장을 조작하여 새로운 문장을 만들기 위한 목적에 따라 여러 개의 텍스트 Generation Model을 사용할 수 있습니다.\n\n1. **Text Generation Model**: 단어, 문장을 조작하여 새로운 문장을 생성하는 Model. 이 Model은 일반적으로 Text-to-Speech (TTS) 및 Text Summarization (TS) 같은 용도로 사용됩니다.\n2. **Language Model**: 단어나 문장을 조작하여 새로운 문장을 생성하는 Model. 이 Model은 일반적으로 Language Translation (LT), Language Understanding (LU), Language Generation (LG) 같은 용도로 사용됩니다.\n3. **Generative Adversarial Network (GAN)**: Text Generation Model과 Language Model의 연합으로, GAN은 새로운 문장을 생성하기 위해 텍스트와 문장을 조작합니다.\n4. **Neural Network-based Models**: Text Generation Model과 Language Model의 연합으로, Neural Network-based Models은 다양한 용도로 사용됩니다.\n\n거대 언어 모델은 다음과 같은 특징을 가지고 있습니다.\n\n* **대형 언어 모델**: 거대 언어 모델은 대규모의 데이터セット을 사용하여 학습하고, 이는 큰 텍스트 생성 capability을 제공합니다.\n* **AI 및 Machine Learning**: 거대 언어 모델은 AI 및Machine Learning에 의해 개발되고, 이러한 기술을 사용하여 더 많은 텍스트를 생성할 수 있습니다.\n* **Flexibility**: 거대 언어 모

pipe에 바로 텍스트를 입력해도 되지만,   
랭체인과의 연동을 위해 `langchain_huggingface` 모듈을 사용하겠습니다.

In [12]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

llm = HuggingFacePipeline(pipeline=pipe, pipeline_kwargs=gen_config)
# Instruct 모델이 아닌 경우

chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)
# Instruct 모델인 경우, tokenizer의 chat template 사용

In [13]:
# Instruct 모델에서는 포맷을 지키지 않으면 제대로 된 결과가 나오지 않음
llm.invoke("에이브러햄 링컨이 누구야?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


' 1964년 7월 5일 태어난 그는 미국의 전 공화국 시대 소설가이자 작가로 유명한 아메리칸 인물입니다.\n\n이브러햄 링컨은 1950년대 후반에 미국에서 일하는 동안, 1952년 11월에 이집트에서 세워진 미국의 첫 번째 우주선인 스타리움 1을 지휘하면서 우주 여행을 시작했습니다. 그는 또한 1953년 9월에 우주 여행을 통해 뉴욕시와 런던시를 방문했습니다. 1960년 10월에는 미국의 첫 번째 우주선인 스톤 1을 지휘하여 1961년 8월 12일에는 그라운드에서 이집트로의 우주선이 도착했습니다. 1966년 11월에 그는 이집트를 떠나 1967년 1월에 미국으로 돌아온 후 14일만에 사망했다. \n\n이브러햄 링컨은 1968년 12월 24일부터 1972년 2월 28일까지 미국의 40대에 belonged을 것으로 간주되었습니다. 1972년 3월 20일부터 1980년 11월 27일까지 그는 50대에 belonged을 것으로 간주되었습니다. 1990년 10월 16일부터 2000년 12월 31일까지 그는 60대에 belonged을 것으로 간주되었습니다. 2003년 11월 29일부터 2010년 10월 18일까지 그는 70대에 belonged을 것으로 간주되었습니다. 2010년 10월 19일부터 2020년 11월 13일까지 그는 80대에 belonged을 것으로 간주되었습니다. 2020년 11월 14일부터 2022년 1월 23일까지 그는 90대에 belonged을 것으로 간주되었습니다. 2022년 1월 24일부터 2023년 1월 26일까지 그는 100대에 belong되었습니다.'

In [14]:
# Tokenizer에 저장된 Template 을 기반으로, 질문 답변을 생성
chat_model.invoke("에이브러햄 링컨이 누구야?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


AIMessage(content='에이브러햄 링컨 (Ebenezer Scrooge, 1812-1871)은 잉글랜드의 시인, 작가이자 시인이자 주도적인 개혁가였습니다. 그는 "A Christmas Carol"라는 책을 출판한 것으로 가장 잘 알려져 있습니다.\n\n그는 1843년 12월 21日に 런던에서 태어난 그는 1850년 8월 7일에 사망했습니다.', additional_kwargs={}, response_metadata={}, id='run-4c4bf532-ad3a-44fe-8ad6-7373df69651c-0')

In [15]:
llm.invoke("Who is Abraham Lincoln?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"?\nAbraham Lincoln was the 16th President of the United States, serving from March 4, 1861 until his assassination in April 1865. He is widely regarded as one of the greatest leaders in American history.\nLincoln was born on February 12, 1809, in a log cabin in Kentucky to Thomas and Nancy Lincoln. His early life was marked by poverty and hard work, but he eventually became a successful lawyer and politician. In 1842, he married Mary Todd, with whom he had four children.\n\nDuring the Civil War, Lincoln faced many challenges, including secessionist states, abolitionists, and slavery. As president, he issued the Emancipation Proclamation in 1863, which declared all slaves in Confederate territory to be free. However, it was not until the passage of the 13th Amendment to the Constitution in 1865 that slavery was officially abolished.\n\nLincoln's leadership during the war led to several key victories, including the Battle of Gettysburg, where he delivered his famous Gettysburg Address. 

In [16]:
chat_model.invoke("Who is Abraham Lincoln?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


AIMessage(content="Abraham Lincoln was the 16th President of the United States, serving from 1861 until his assassination in 1865. He is widely regarded as one of the most influential leaders in American history.\n\nLincoln was born on February 12, 1809, in a log cabin in Kentucky to Thomas and Nancy Lincoln. He grew up in poverty and had very little formal education, but he was an avid reader and developed a strong interest in law and politics. He studied law in Springfield, Illinois, where he became a licensed attorney at age 21.\n\nIn 1832, Lincoln married Mary Todd, and they had four children together. During the Mexican-American War (1846-1848), Lincoln served in the Illinois state legislature and later as a member of the U.S. House of Representatives. In 1858, he ran for the U.S. Senate against Stephen Douglas, a prominent Democrat, and won, becoming a national figure due to his strong oratory skills.\n\nThe election of 1860 sparked controversy when several Southern states secede

불러온 모델의 종류에 따라, 토크나이저에서 Special Tokens와 Chat Template를 확인할 수 있습니다.

In [7]:
tokenizer.all_special_tokens

['<|begin_of_text|>', '<|eot_id|>']

In [ ]:
tokenizer.eos_token, tokenizer.bos_token, tokenizer.pad_token

In [ ]:
print(tokenizer.chat_template)

OpenAI에서 배웠던 포맷의 메시지는    
토크나이저를 통해 변환할 수 있습니다.

In [ ]:
chat = [
    { "role": "user", "content": "한국 프로야구 리그인 KBO에 대해 설명해 주세요." },
]
prompt = tokenizer.apply_chat_template(chat,
                                       tokenize=False,
                                       add_generation_prompt= True) #False)

# add_generation_prompt : 입력 텍스트 뒤에 generation prompt를 추가할지 여부

print(prompt)

채팅 모델에서는 아래와 같이 토크나이저를 연결해야 하나,    
ChatHuggingFace에서는 그대로 실행해도 됩니다.

In [ ]:
summarize_instruction ='''아래의 글을 읽고, 200자 이내의 요약문을 작성하세요.'''

example_news='''LG 트윈스가 6월 14일부터 16일까지 잠실야구장에서 열리는 롯데 자이언츠와 주말 홈 3연전에서 승리 기원 시구 및 '잔망루피 데이'를 진행한다.

먼저 14일에는 '피지컬: 100 시즌2'에 출연한 전 핸드볼 선수 박하얀이 시구, 프로그램 우승자 아모띠가 시타를 각각 진행한다. 두 사람은 "선수들 모두 다치지 않고, 좋은 경기를 펼치면 좋겠다. LG 트윈스가 좋은 성적을 거둘 수 있도록 끝까지 열심히 응원하겠다"고 소감을 전했다. 이날 경기 전에는 박해민 선수의 개인 통산 1500안타와 2000루타 달성을 기념하는 KBO 시상식도 열릴 예정이다.

이어 15일에는 이건태 소방관의 시구와 농심 '배홍동' 마스코트의 시타가 있다. 이건태 소방관은 구조대원으로서 화재 진압 및 여러 구조 현장에서 활동하고 있다. 특히 수중 수색 기술을 개발, 교육하는 등 대한민국 구조 활동에 중요한 역할을 하고 있다. 이날 경기 전에는 신민재, 문성주의 팬 사인회와 김현수의 개인 통산 2000경기 출장 달성을 기념하는 KBO 시상식이 진행된다.


아이돌그룹 '더보이즈'의 멤버 현재. /사진=LG 트윈스 제공
끝으로 16일에는 아이돌그룹 '더보이즈'의 멤버 현재가 승리 기원 시구를 진행한다. '더보이즈'는 오는 7월 월드투어를 통해 팬들을 만날 예정이다. 최근 두 번째 정규 앨범인 '판타지(PHANTASY)' Pt.3 '러브레터(Love Letter)'를 발매하고, 음원과 음반, 글로벌 차트 등에서 큰 성과를 거두고 있다. 현재는 "데뷔 후 첫 시구 도전이라 떨리는 마음도 있지만 설레는 마음이 더 크다. 자신 있게 시구하고, 선수들이 좋은 경기 펼칠 수 있도록 열심히 응원하겠다"고 전했다.

아울러 LG 트윈스는 '잔망루피 데이'를 맞아 '잔망루피' 스페셜 티켓과 내야 광장 포토존을 운영하고, 2024시즌 '잔망루피' 콜라보 신상품을 출시한다. 출시 상품은 유니폼, 마킹 키트, 모자, 봉제 인형과 키링, 응원 타월, 부채, 부적 키링 3종과 마그넷 세트 등이며, 이를 기념해 10% 할인 프로모션을 진행한다. '잔망루피'는 지난 2021년 LG트윈스의 신입 응원단원으로 합류했으며, 다양한 협업 활동을 통해 팬들의 사랑을 받고 있다.

한편 이번 주말 홈 3연전 포토 카드의 주인공은 오지환과 '잔망루피'다. LG 트윈스 홈 경기 이벤트 진행에 대한 자세한 내용은 LG트윈스 홈페이지와 모바일앱, SNS 계정을 통해 확인할 수 있다.
'''

prompt = [
    {'role':'system','content':summarize_instruction},
    {'role':'user','content':f'Context: {example_news}'}]

llm.invoke(tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True))

In [ ]:
prompt = [
    {'role':'system','content':'''당신은 전문적인 QA 봇입니다.
아래의 Context와 질문을 읽고, 이에 대한 답변을 작성하세요.
Context에서 확인할 수 있는 내용만을 사용하여 답변하고, 답변을 지어내지 마세요.'''},
    {'role':'user','content':f'''
Context={example_news}

질문: 왜 잔망루피가 LG 트윈스 포토카드에 있어?

답변:'''}]

print(llm.invoke(tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)))

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', '{instruction}'),
        ('user', '{context}')
    ]
)

llm_chain = chat_template | chat_model | StrOutputParser()

In [ ]:
print(llm_chain.invoke({'instruction':summarize_instruction, 'context':example_news}))